# Simulační studie: Odhad rozptylu výběrových kvantilů

Tento notebook obsahuje kód pro generování dat a porovnání metod odhadu rozptylu kvantilů (Taylor, Plug-in, Bootstrap). Výstupem jsou grafy Relative Bias a Coverage Probability.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_theme(style="whitegrid")
plt.rcParams.update({'figure.figsize': (10, 6), 'figure.dpi': 100})

# Configuration
np.random.seed(42)
Ns = [30, 100, 1000]
Ps = [0.50, 0.95, 0.99]
n_mc = 1000 # Monte Carlo replications
n_boot = 200 # Bootstrap replications
mu, sigma = 0, 1.0 # Log-normal parameters

In [ ]:
# True values helpers
def get_true_quantile(p, mu, sigma):
    return stats.lognorm.ppf(p, s=sigma, scale=np.exp(mu))

def get_true_pdf(x, mu, sigma):
    return stats.lognorm.pdf(x, s=sigma, scale=np.exp(mu))

true_quatiles = {p: get_true_quantile(p, mu, sigma) for p in Ps}
true_densities = {p: get_true_pdf(true_quatiles[p], mu, sigma) for p in Ps}

In [ ]:
results = []

print("Starting simulation...")

for n in Ns:
    print(f"  Running for n={n}...")
    
    mc_storage = {p: {'q_hat': [], 'var_oracle': [], 'var_plugin': [], 'var_boot': []} for p in Ps}
    
    for _ in range(n_mc):
        # Generate data
        data = stats.lognorm.rvs(s=sigma, scale=np.exp(mu), size=n)
        
        try:
            kde = stats.gaussian_kde(data)
        except:
            kde = None
            
        for p in Ps:
            # Point estimate
            q_hat = np.quantile(data, p)
            mc_storage[p]['q_hat'].append(q_hat)
            
            # 1. Oracle Taylor
            f_true = true_densities[p]
            var_oracle = (p * (1 - p)) / (n * f_true**2)
            mc_storage[p]['var_oracle'].append(var_oracle)
            
            # 2. Plug-in Taylor
            if kde is not None:
                f_hat = kde(q_hat)[0]
            else:
                f_hat = 1e-10
            if f_hat < 1e-10: f_hat = 1e-10
            
            var_plugin = (p * (1 - p)) / (n * f_hat**2)
            mc_storage[p]['var_plugin'].append(var_plugin)
            
            # 3. Bootstrap
            boot_qs = []
            for _ in range(n_boot):
                sample_b = np.random.choice(data, size=n, replace=True)
                boot_qs.append(np.quantile(sample_b, p))
            var_boot = np.var(boot_qs, ddof=1)
            mc_storage[p]['var_boot'].append(var_boot)
            
    # Process results
    for p in Ps:
        true_q = true_quatiles[p]
        q_hats = np.array(mc_storage[p]['q_hat'])
        
        var_mc = np.var(q_hats, ddof=1)
        se_mc = np.sqrt(var_mc)
        
        # Mean estimated SE
        mean_se_oracle = np.mean(np.sqrt(mc_storage[p]['var_oracle']))
        mean_se_plugin = np.mean(np.sqrt(mc_storage[p]['var_plugin']))
        mean_se_boot = np.mean(np.sqrt(mc_storage[p]['var_boot']))

        # Coverage
        cov_oracle = 0; cov_plugin = 0; cov_boot = 0
        z = 1.96
        
        for i in range(n_mc):
            qh = mc_storage[p]['q_hat'][i]
            if (qh - z*np.sqrt(mc_storage[p]['var_oracle'][i]) <= true_q <= qh + z*np.sqrt(mc_storage[p]['var_oracle'][i])): cov_oracle += 1
            if (qh - z*np.sqrt(mc_storage[p]['var_plugin'][i]) <= true_q <= qh + z*np.sqrt(mc_storage[p]['var_plugin'][i])): cov_plugin += 1
            if (qh - z*np.sqrt(mc_storage[p]['var_boot'][i]) <= true_q <= qh + z*np.sqrt(mc_storage[p]['var_boot'][i])): cov_boot += 1
            
        results.append({
            'n': n, 'p': p,
            'SE_MC': se_mc,
            'RB_Oracle': (mean_se_oracle - se_mc) / se_mc,
            'RB_Plugin': (mean_se_plugin - se_mc) / se_mc,
            'RB_Boot': (mean_se_boot - se_mc) / se_mc,
            'Cov_Oracle': cov_oracle / n_mc,
            'Cov_Plugin': cov_plugin / n_mc,
            'Cov_Boot': cov_boot / n_mc
        })

df = pd.DataFrame(results)
print("Simulation done.")

In [ ]:
# Plot Relative Bias
df_melt_bias = df.melt(id_vars=['n', 'p'], value_vars=['RB_Oracle', 'RB_Plugin', 'RB_Boot'], 
                       var_name='Method', value_name='Relative Bias')
df_melt_bias['Method'] = df_melt_bias['Method'].str.replace('RB_', '')

g = sns.FacetGrid(df_melt_bias, col="p", sharey=False, height=5, aspect=0.8)
g.map_dataframe(sns.lineplot, x="n", y="Relative Bias", hue="Method", style="Method", markers=True)
g.add_legend()
g.set(xscale="log")
plt.show()

In [ ]:
# Plot Coverage Probability
df_melt_cov = df.melt(id_vars=['n', 'p'], value_vars=['Cov_Oracle', 'Cov_Plugin', 'Cov_Boot'], 
                      var_name='Method', value_name='Coverage')
df_melt_cov['Method'] = df_melt_cov['Method'].str.replace('Cov_', '')

g = sns.FacetGrid(df_melt_cov, col="p", sharey=True, height=5, aspect=0.8)
g.map_dataframe(sns.lineplot, x="n", y="Coverage", hue="Method", style="Method", markers=True)
for ax in g.axes.flat:
    ax.axhline(0.95, ls='--', color='gray', alpha=0.5)
g.add_legend()
g.set(xscale="log")
plt.show()